In [1]:
import numpy as np
import torch
import torch.nn as nn 
import torch.nn.functional as F
import pandas as pd
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../plant_pathology/scripts/')
from dataset.dataloaders import GetLoaders
from dataset.dataset import ReadCSV
import os
from train.loss import loss_fn
from model.resnet32 import get_model
from model.vgg16 import get_vgg16
from model.incep_v1 import get_incep_v1
from train.train_script import train, eval_dataset
import torchvision
from dataset.resize_images import resize_images
import glob
# from kaggle.api.kaggle_api_extended import KaggleApi
import pickle
from metrics.metrics import generate_metrics
import matplotlib.pyplot as plt
import torchvision
from dataset.dataset import get_class_weight
from functools import partial

In [ ]:
# resize_images(folder_path='../../plant_path_data/train_images/', new_path='../../plant_path_data/train_images_224/', img_size=224)

In [ ]:
temp = ReadCSV('../../plant_path_data/train.csv', '../../plant_path_data/train_images_224/')
temp.read_csv()
temp.check_paths()
temp.encode_labels()
temp.one_hot_encoder()
temp.split_datasets()

In [ ]:
# temp.check_image_valids()

In [ ]:
data_object = temp.get_object()
loader_object = GetLoaders(data_object.train, data_object.val, data_object.test)
loader_object.create_sets()
loader_object.create_loaders(batch_size=256)
train_loader, val_loader, test_loader = loader_object.return_loaders()

In [ ]:
wandb_dict = {
    'model_name' : 'res32',
    'dataset_name' : 'plant_pathology',
    'epochs' : 50,
    'batch_size' : 256,
    'learning_rate' : 0.001,
    'optimizer' : 'Adam',
    'weights_path' : '../../weights/res32_horizontal_flip_random_rotate/',
    'horizontal_flip' : True,
    'random_rotation' : True,
    'normalize' : True,
    'fixed_metrics' : True
}

In [ ]:
train(get_model(12), train_loader, val_loader, 50, wandb_dict=wandb_dict, save_path='../../weights/res32_horizontal_flip_random_rotate_norm/')

## create worst performing classes report

In [ ]:
data_object = temp.get_object()
loader_object = GetLoaders(data_object.train, data_object.val, data_object.test)
loader_object.create_sets()
loader_object.create_loaders(shuffle_train=False, batch_size=256)
train_loader, val_loader, test_loader = loader_object.return_loaders()

In [ ]:
# eval_dataset(get_model(12), train_loader, wandb_dict=wandb_dict, save_path='../../weights/res32_horizontal_flip_random_rotate_norm/21.pth')
loss, return_preds = eval_dataset(get_model(12), train_loader, weights_path = '../../weights/res32_horizontal_flip_random_rotate_norm/21.pth', return_preds = True)

In [ ]:
train_labels = torch.Tensor(train_loader.dataset.labels).long()

In [ ]:
print(return_preds)


In [ ]:
data_object.label_encoder.inverse_transform([0])[0]

In [ ]:
def calculate_loss_classwise(preds, labels, num_classes=12):
    losses = []
    amount_of_samples = []
    label_to_class = []
    for i in range(num_classes):
        loss = loss_fn(preds[torch.where(labels == i)[0]], labels[torch.where(labels == i)[0]])
        losses.append(loss.item())
        amount_of_samples.append(len(torch.where(labels == i)[0]))
        label_to_class.append(data_object.label_encoder.inverse_transform([i])[0])
    return losses, amount_of_samples, label_to_class

In [ ]:
loss, num_samples, label_to_class = calculate_loss_classwise(return_preds, train_labels)

In [ ]:
analysis_df = pd.DataFrame({'loss' : loss, 'num_samples' : num_samples, 'label_to_class' : label_to_class})

In [ ]:
analysis_df

In [ ]:
plt.bar(analysis_df['loss'].values, analysis_df['num_samples'].values)

In [ ]:
X = [x for x in range(12)]
num_samples_ = analysis_df['num_samples'].values / np.sum(analysis_df['num_samples'].values)
loss_ = analysis_df['loss'].values
  
X_axis = np.arange(len(X)) 
  
plt.bar(X_axis - 0.2, num_samples_, 0.4, label = 'Samples') 
plt.bar(X_axis + 0.2, loss_, 0.4, label = 'Losses') 
  
plt.xticks(X_axis, X) 
# plt.xlabel("Class") 
# plt.ylabel("Number of Students") 
# plt.title("Number of Students in each group") 
plt.legend() 
plt.show()

In [ ]:
# classes with max loss -> 0, 2, 5, 7, 8, 10, 11

In [ ]:
# model = get_model(12)
# model.load_state_dict(torch.load('../../weights/res32_horizontal_flip_random_rotate_norm/33.h5'))

In [ ]:
data_object = temp.get_object()
loader_object = GetLoaders(data_object.train, data_object.val, data_object.test)
loader_object.create_sets()
loader_object.create_loaders(shuffle_train=False, batch_size=256)
train_loader, val_loader, test_loader = loader_object.return_loaders()

In [ ]:
# Eval on test set
eval_dataset(get_model(12), val_loader, '../../weights/res32_horizontal_flip_random_rotate_norm/23.pth')

In [ ]:
# Eval on test set
eval_dataset(get_model(12), test_loader, '../../weights/res32_horizontal_flip_random_rotate_norm/23.pth')

In [ ]:
# Eval on test set
loss, preds = eval_dataset(get_model(12), train_loader, '../../weights/res32_horizontal_flip_random_rotate_norm/23.pth', return_preds=True)

In [ ]:
c_m, precision, recall = generate_metrics(preds, torch.Tensor(train_loader.dataset.labels).long(), num_classes=12)

## Training with selected classes with training

In [ ]:
temp = ReadCSV('../../plant_path_data/train.csv', '../../plant_path_data/train_images_224/')
temp.read_csv()
temp.check_paths()
temp.encode_labels()
temp.one_hot_encoder()
temp.split_datasets()

In [ ]:
data_object = temp.get_object()
loader_object = GetLoaders(data_object.train, data_object.val, data_object.test)
loader_object.create_sets()
loader_object.create_loaders(batch_size=256)
train_loader, val_loader, test_loader = loader_object.return_loaders()

In [ ]:
wandb_dict = {
    'model_name' : 'res50',
    'dataset_name' : 'plant_pathology',
    'epochs' : 50,
    'batch_size' : 256,
    'learning_rate' : 0.001,
    'optimizer' : 'Adam',
    'weights_path' : '../../weights/res50_horizontal_flip_random_brightness_contrast/',
    'horizontal_flip' : True,
    'random_rotation' : True,
    'normalize' : True,
    'color_jitter' : True,
    # 'selected_classes' : [0, 2, 5, 7, 8, 10, 11]
}

In [ ]:
model = torchvision.models.resnet50(pretrained=False)
model.fc = nn.Linear(2048, 12)

In [ ]:
train(model, train_loader, val_loader, 50, wandb_dict=wandb_dict, save_path='../../weights/res50_horizontal_flip_random_brightness_contrast/')

## Training with augmentation and label smoothing

In [ ]:
temp = ReadCSV('../../plant_path_data/train.csv', '../../plant_path_data/train_images_224/')
temp.read_csv()
temp.check_paths()
temp.encode_labels()
temp.one_hot_encoder()
temp.split_datasets()

In [ ]:
data_object = temp.get_object()
loader_object = GetLoaders(data_object.train, data_object.val, data_object.test)
loader_object.create_sets()
loader_object.create_loaders(batch_size=256)
train_loader, val_loader, test_loader = loader_object.return_loaders()

In [ ]:
wandb_dict = {
    'model_name' : 'res32',
    'dataset_name' : 'plant_pathology',
    'epochs' : 50,
    'batch_size' : 256,
    'learning_rate' : 0.001,
    'optimizer' : 'Adam',
    'weights_path' : '../../weights/res32_horizontal_flip_random_brightness_contrast_smoothing/',
    'horizontal_flip' : True,
    'random_rotation' : True,
    'normalize' : True,
    'color_jitter' : True,
    'label_smoothing' : 0.1,
    # 'selected_classes' : [0, 2, 5, 7, 8, 10, 11]
}

In [ ]:
train(get_model(12), train_loader, val_loader, 50, wandb_dict=wandb_dict, save_path='../../weights/res50_horizontal_flip_random_brightness_contrast_smoothing/')

## Training with label smoothing, augmentation and class weights and lr schuduler

In [ ]:
temp = ReadCSV('../../plant_path_data/train.csv', '../../plant_path_data/train_images_224/')
temp.read_csv()
temp.check_paths()
temp.encode_labels()
temp.one_hot_encoder()
temp.split_datasets()
data_object = temp.get_object()
loader_object = GetLoaders(data_object.train, data_object.val, data_object.test)
loader_object.create_sets()
loader_object.create_loaders(batch_size=512)
train_loader, val_loader, test_loader = loader_object.return_loaders()

In [ ]:
data_object = temp.get_object()
loader_object = GetLoaders(data_object.train, data_object.val, data_object.test)
loader_object.create_sets()
loader_object.create_loaders(batch_size=512)
train_loader, val_loader, test_loader = loader_object.return_loaders()

In [ ]:
class_weights = get_class_weight(train_loader.dataset.labels)

In [ ]:
wandb_dict = {
    'model_name' : 'res32',
    'dataset_name' : 'plant_pathology',
    'epochs' : 50,
    'batch_size' : 512,
    'learning_rate' : 0.001,
    'optimizer' : 'Adam',
    'weights_path' : '../../weights/res32_horizontal_flip_random_brightness_contrast_smoothing_class_weight_lrschedule_v1/',
    'horizontal_flip' : True,
    'random_rotation' : True,
    'normalize' : True,
    'color_jitter' : True,
    'class_weights' : [True, class_weights],
    'StepLR': True
    # 'selected_classes' : [0, 2, 5, 7, 8, 10, 11]
}

In [ ]:
train(get_model(12), 
      train_loader, 
      val_loader, 
      50, 
      wandb_dict=wandb_dict, 
      load_model='../../weights/res32_horizontal_flip_random_brightness_contrast_smoothing_class_weight_v2/49.pth', 
      class_weights=class_weights, save_path='../../weights/res32_horizontal_flip_random_brightness_contrast_smoothing_class_weight_lrschedule_v1/')

In [ ]:
import torch.optim as optim
m = get_model(12)

In [ ]:
optimizer = optim.Adam(m.parameters(), lr=0.001)

In [ ]:
optimizer.state_dict()['param_groups'][0]['lr']

## Inception model training

In [2]:
temp = ReadCSV('../../plant_path_data/train.csv', '../../plant_path_data/train_images_224/')
temp.read_csv()
temp.check_paths()
temp.encode_labels()
temp.one_hot_encoder()
temp.split_datasets()
data_object = temp.get_object()
loader_object = GetLoaders(data_object.train, data_object.val, data_object.test)
loader_object.create_sets(train_augment = True)
loader_object.create_loaders(num_workers = 10, batch_size=512)
train_loader, val_loader, test_loader = loader_object.return_loaders()

In [3]:
wandb_dict = {
    'model_name' : 'incep_v1',
    'dataset_name' : 'plant_pathology',
    'epochs' : 100,
    'batch_size' : 512,
    'num_workers' : 10,
    'learning_rate' : 0.001,
    'optimizer' : 'Adam',
    'weights_path' : '../../weights/incep_v1_hflip_rrotate_jitter_all_sh_0_7/',
    'normalize' : True,
    'horizontal_flip' : True,
    'random_rotation' : True,
    'color_jitter' : True,
    'brightness' : 0.5,
    'contrast' : 0.5,
    'saturation' : 0.3,
    'hue' : 0.3,
}

In [4]:
train_component = partial(train, model=get_incep_v1(12), train_loader=train_loader, val_loader=val_loader, epochs=100, wandb_dict=wandb_dict)

In [5]:
train_component(save_path='../../weights/incep_v1_hflip_rrotate_jitter_all_sh_0_7/')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yashchks87. Use `wandb login --relogin` to force relogin


100%|██████████| 36/36 [00:27<00:00,  1.32batch/s, loss=1.93]


train loss: 2.0094438791275024


100%|██████████| 1/1 [00:00<00:00,  1.10batch/s, loss=1.95]


val loss: 1.9494487047195435
Weight saved for epoch 0.


...

val loss: 0.5581945776939392
Weight saved for epoch 98.


100%|██████████| 36/36 [00:27<00:00,  1.32batch/s, loss=0.564]


train loss: 0.5322755244043138


100%|██████████| 1/1 [00:00<00:00,  1.05batch/s, loss=0.615]


val loss: 0.6154524683952332
Weight saved for epoch 99.


## Inception with all augmentation and also with Cut out

In [7]:
cut_out_dict = {
    'size_of_cutout' : 24,
    'num_of_cutout_holes' : 4
}

In [8]:
temp = ReadCSV('../../plant_path_data/train.csv', '../../plant_path_data/train_images_224/')
temp.read_csv()
temp.check_paths()
temp.encode_labels()
temp.one_hot_encoder()
temp.split_datasets()
data_object = temp.get_object()
loader_object = GetLoaders(data_object.train, data_object.val, data_object.test)
loader_object.create_sets(train_augment = True, cut_out_params = cut_out_dict)
loader_object.create_loaders(num_workers = 10, batch_size=128)
train_loader, val_loader, test_loader = loader_object.return_loaders()

In [9]:
wandb_dict = {
    'model_name' : 'incep_v1',
    'dataset_name' : 'plant_pathology',
    'epochs' : 50,
    'batch_size' : 256,
    'num_workers' : 10,
    'learning_rate' : 0.001,
    'optimizer' : 'Adam',
    'weights_path' : '../../weights/incep_v1_hflip_rotate_jitter_cut_out_v3/',
    'normalize' : True,
    'horizontal_flip' : True,
    'random_rotation' : True,
    'color_jitter' : True,
    'brightness' : 0.5,
    'contrast' : 0.5,
    'saturation' : 0.5,
    'hue' : 0.5,
    'cut_out_dict' : cut_out_dict
}

In [10]:
train_component = partial(train, model=get_incep_v1(12), train_loader=train_loader, val_loader=val_loader, epochs=50, wandb_dict=wandb_dict)

In [11]:
train_component(load_model = '../../weights/incep_v1_hflip_rotate_jitter_cut_out_v2/49.pth', save_path='../../weights/incep_v1_hflip_rotate_jitter_cut_out_v3/')

train_loss,█▇▆▆▇▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
train_prec_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_prec_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_prec_10,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_prec_11,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_prec_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_prec_3,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_prec_4,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_prec_5,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_prec_6,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_prec_7,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


100%|██████████| 143/143 [00:24<00:00,  5.75batch/s, loss=0.607]


train loss: 0.5805868885733865


100%|██████████| 2/2 [00:00<00:00,  2.72batch/s, loss=0.738]


val loss: 0.6681874096393585
Weight saved for epoch 0.


...

val loss: 0.5855647623538971
Weight saved for epoch 48.


100%|██████████| 143/143 [00:24<00:00,  5.83batch/s, loss=0.337]


train loss: 0.4702458381652832


100%|██████████| 2/2 [00:00<00:00,  2.78batch/s, loss=0.592]


val loss: 0.6108399033546448
Weight saved for epoch 49.
